# Phase 2 (recommendation links) - Ongoing

In [2]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver 
import time
import seaborn as sns 
from Hemnet import preprocessing,hemnet_generator, dep_filter,pct_change_metric
from Scraper import url_extractor , scraper2
from bs4 import BeautifulSoup
import requests
from itertools import chain 
import numpy as np 
import re  
from flask import Flask, request,render_template
import pickle

## NEXT: 
- 1 hyperlinks
- create a functions to calcualte precentage of change by area 
- 2 send by email


In [9]:
######
# 1 Extract starter info
######

def url_extractor2(area ='sundbyberg',keys = None, min_year = 1980):
    print('INITIAL EXTRACTION...')
    chrome_browser = webdriver.Chrome('/Users/Tabe/Desktop/pythonprojects/automation/chromedriver')
    chrome_browser.get('https://www.hemnet.se/bostader?item_types%5B%5D=bostadsratt')
    chrome_browser.maximize_window() 
    try:
        cookie_button = chrome_browser.find_element_by_xpath('/html/body/div[9]/div/div/div[2]/div/button')
        cookie_button.click()
        time.sleep(2)
    except NoSuchElementException: 
        cookie_button = chrome_browser.find_element_by_xpath('/html/body/div[10]/div/div/div[2]/div/button')
        cookie_button.click()
    #/html/body/div[8]/div/div/form/div[1]/div[2]/label[3]/span[2]

    try:
    # Area 
        box = chrome_browser.find_element_by_xpath('//*[@id="area-search-input-box"]')
        box.clear()
        time.sleep(2)
        box.send_keys(area)
        time.sleep(2)
        dropdown = chrome_browser.find_element_by_xpath('//*[@id="search-location-dropdowns"]/div/div[2]/div[1]/ul/li[1]')
        dropdown.click() # dropdown 
        time.sleep(2)
    # Bo area
        min_area = chrome_browser.find_element_by_xpath('//*[@id="search_living_area_min"]/option[6]')
        max_area =  chrome_browser.find_element_by_xpath('//*[@id="search_living_area_max"]/option[11]') 

        time.sleep(2)
        min_area.click() # dropdown 
        time.sleep(2)
        max_area.click() # dropdown 
        time.sleep(2)
    
    # Search price max : 2650 000

        op3 =  chrome_browser.find_element_by_xpath('//*[@id="search_price_max"]/option[15]')
        op3.click() # dropdown 
        time.sleep(2)
    
        if keys != None:
            search_keywords =  chrome_browser.find_element_by_xpath('//*[@id="search_keywords"]')
            search_keywords.send_keys(keys)
            time.sleep(1)
        
        op4 =  chrome_browser.find_element_by_xpath('//*[@id="new_search"]/button')
        op4.click() # dropdown 
    
        construction_year =  chrome_browser.find_element_by_xpath('//*[@id="search_construction_year_min"]')
        construction_year.send_keys(min_year)
        time.sleep(1)

        op5 = chrome_browser.find_element_by_xpath('//*[@id="search_fee_max"]/option[10]')
        op5.click() # dropdown 
        time.sleep(2)
        cta = chrome_browser.find_element_by_xpath('//*[@id="new_search"]/div[3]/div/button')
        cta.click() # dropdown 
        time.sleep(3)
        current_url = chrome_browser.current_url
        print('DONE WITH INITIAL EXTRACTION')
        return current_url
    
    except ElementClickInterceptedException as err: 
        print(f'You got this error{err}')


In [10]:
#####
#2 Initial scrape
#####

def scraperx(current_url, relevant_only = 'yes' , sold_age = '6m' , loan_limit = 2985000):

    print('STEP 1: SCRAPING...')
    res = requests.get(current_url)
    soup = BeautifulSoup(res.content,'html.parser')
    street  =  [i.text for i in soup.find_all('h2', class_ ="listing-card__street-address qa-listing-title" )]
    location  =  [i.text for i in soup.find_all('span', class_ ="listing-card__location-name" )]

    prices_ =  [ i.text for i in soup.find_all('div', class_ ="listing-card__attributes-row" ) ]
    prices  = [prices_[i] for i in range(0,len(prices_),2)]
    links =  []
    for link in soup.findAll('a', attrs={'href': re.compile("^https://www.hemnet")}):
        links.append(link.get('href') )
    d = {'street':street , 'location':location , 'prices':prices , 'links':links }
    df = pd.DataFrame(d)
    
    df['start_price'] = df.prices.apply(lambda x: x.split('\n\n')[1] )
    df['ap_size'] = df.prices.apply(lambda x: x.split('\n\n')[2] )
    df['number_of_rooms'] = df.prices.apply(lambda x: x.split('\n\n')[3] )
    df.drop('prices', axis = 1 ,inplace = True )
    
    print('STEP 2: GENERATING NEW FEATURES...')
    df['ap_size'] = df['ap_size'].apply(lambda x: float(x.split()[0] if ',' not in x else float(x.split()[0].replace(',', '.'))) )
    df['start_price'] = df.start_price.apply(lambda col: int(''.join([i for i in col if i.isnumeric()] )))
    df['number_of_rooms'] = df.number_of_rooms.apply(lambda col: int(''.join([i for i in col if i.isnumeric()] )))
    df['city-kommun'] = df.location.apply(lambda x: str(x.split()[-1:] )[1:-1].strip("''") )
    df['area'] = df.location.apply(lambda x: x.split(',')[0].strip('\n '))
    df['street'] = df['street'].apply(lambda x: x.strip('\n ') )
    df.drop('location', axis= 1 , inplace=True )
    
    print('STEP 3: OBTAINING HISTORICAL DATA TO CALCULATE PREDICTED PRICE...')
    area_codes = {'järfälla_code': 17951,
                  'sollentuna_code' :18027,
                  'solna_code':18028,
                  'sundbyberg_code': 18042,
                  'stockholms_län_code': 18031,
                  'vällingby_code' : 473464, 
                  'söder_code' :898472, 
                  'bromma': 898740
                 
                 }

    area = df['city-kommun'][0] # get the area
    pat = re.compile(r"\b(\w*{}\w*)\b".format(area.lower()))

    area_code = 0
    for i in area_codes.keys(): 
        if pat.search(i):
            area_code+= area_codes[i]
    
    history = hemnet_generator(sold_age = sold_age, area_code = area_code  ,num_pages = pages_size, relevant_size= True)
    
    print('STEP 4: GENERATING LAST FEATURES...')
    mean_price_change = history.price_change.mean()
    mean_price_per_m2 = history.pris_per_m2.mean()

    df['predicted_price'] = df.start_price +  (df.start_price  * (mean_price_change/100) )
    df['price_per_m2'] = df.start_price /  df.ap_size
    df['label'] = df.predicted_price.apply(lambda x: 'possible' if x <= loan_limit else 'less possible')
    df['expected_price'] = df.ap_size * mean_price_per_m2

    
    # re order cols  
    df = df[['city-kommun','area','street','label','ap_size','number_of_rooms','start_price','predicted_price','expected_price','price_per_m2','links']]
    
    if relevant_only == 'yes' : 
        print('DONE')
        df_new = df[df['label']=='possible'].sort_values('predicted_price') 
        return df_new
    elif relevant_only == 'no' : 
        print('DONE')
        return df.sort_values('predicted_price') 
    else:
        raise KeyError 

SyntaxError: invalid syntax (<ipython-input-10-fab2582b6b31>, line 2)

In [42]:
#########
# Run the function
#########
extraction = url_extractor2(area ='solna', min_year = 1980 )
df = scraperx(current_url = extraction , relevant_only = 'no'  , sold_age = '3m' , loan_limit = 3000000)

INITIAL EXTRACTION...
DONE WITH INITIAL EXTRACTION
STEP 1: SCRAPING...
STEP 2: GENERATING NEW FEATURES...
STEP 3: OBTAINING HISTORICAL DATA TO CALCULATE PREDICTED PRICE...
STEP 4: GENERATING LAST FEATURES...
DONE


In [43]:
df

,city-kommun,area,street,label,ap_size,number_of_rooms,start_price,predicted_price,expected_price,price_per_m2,links
11,Solna,Solna Ulriksdal,Paradgatan 51,possible,44.0,2,2495000,2.868509e+06,2.824962e+06,56704.545455,https://www.hemnet.se/bostad/lagenhet-2rum-sol...
3,Solna,Västra Skogen,Armégatan 36D,possible,48.0,2,2595000,2.983479e+06,3.081777e+06,54062.500000,https://www.hemnet.se/bostad/lagenhet-2rum-vas...
5,Solna,Arenastaden,"Evenemangsgatan 34, 4tr",possible,49.0,2,2595000,2.983479e+06,3.145980e+06,52959.183673,https://www.hemnet.se/bostad/lagenhet-2rum-are...
7,Solna,Frösunda,Signe Tillischgatan 9,less possible,42.0,2,2695000,3.098450e+06,2.696555e+06,64166.666667,https://www.hemnet.se/bostad/lagenhet-2rum-fro...
8,Solna,Arenastaden,"Evenemangsgatan 30, 14tr",less possible,43.0,2,2695000,3.098450e+06,2.760758e+06,62674.418605,https://www.hemnet.se/bostad/lagenhet-2rum-are...
12,Solna,Huvudsta,"Lundagatan 1B, ½tr",less possible,52.0,25,2725000,3.132941e+06,3.338592e+06,52403.846154,"https://www.hemnet.se/bostad/lagenhet-2,5rum-h..."
4,Solna,SOLNA,"Honnörsgatan 5, HÖGST UPP!",less possible,53.0,2,2795000,3.213420e+06,3.402795e+06,52735.849057,https://www.hemnet.se/bostad/lagenhet-2rum-sol...
9,Solna,Solna Arenastaden,"Evenemangsgatan 40, vån 18",less possible,43.0,2,2795000,3.213420e+06,2.760758e+06,65000.000000,https://www.hemnet.se/bostad/lagenhet-2rum-sol...
13,Solna,Solna Järvastaden,Fridensborgsvägen 148,less possible,46.0,2,2795000,3.213420e+06,2.953369e+06,60760.869565,https://www.hemnet.se/bostad/lagenhet-2rum-sol...
14,Solna,Solna,Våtmarksvägen 62,less possible,52.0,2,2795000,3.213420e+06,3.338592e+06,53750.000000,https://www.hemnet.se/bostad/lagenhet-2rum-sol...


In [13]:
zones = [18027,18028,18042,17951]
for area_code in zones:
    pct_change_metric (area_code , num_pages= 50, metric = 'pris_per_m2')

Calculating param 12m...
Calculating param 6m...
Calculating param 3m...
The pris_per_m2 has increased by 10.0%
{'12m': 51112.070788530466, '6m': 50862.59463722398, '3m': 56175.06469002696}
Calculating param 12m...
Calculating param 6m...
Calculating param 3m...
The pris_per_m2 has increased by 2.0%
{'12m': 59129.247181628394, '6m': 59257.265227447955, '3m': 60496.30291005291}
Calculating param 12m...
Calculating param 6m...
Calculating param 3m...
The pris_per_m2 has increased by 4.0%
{'12m': 58389.64387464387, '6m': 59351.68048780488, '3m': 60975.81263157895}
Calculating param 12m...
Calculating param 6m...
Calculating param 3m...
The pris_per_m2 has increased by 3.0%
{'12m': 36341.223544973545, '6m': 36889.393867924526, '3m': 37496.39361702128}


In [340]:
links = []
for link in soup.findAll('a', attrs={'href': re.compile("^https://www.hemnet")}):
    links.append(link.get('href') )

In [341]:
links

['https://www.hemnet.se/bostad/lagenhet-3rum-tureberg-sollentuna-kommun-engelbrekts-vag-73-c,-bv-17067632',
 'https://www.hemnet.se/bostad/lagenhet-2rum-edsangen-sollentuna-kommun-frestavagen-32-17066611',
 'https://www.hemnet.se/bostad/lagenhet-2rum-centrala-sollentuna-sollentuna-kommun-attundagrand-5-17066877',
 'https://www.hemnet.se/bostad/lagenhet-2rum-tureberg-sollentuna-kommun-turebergs-alle-30-5-tr-17064758',
 'https://www.hemnet.se/bostad/lagenhet-2rum-edsangen-sollentuna-kommun-frestavagen-28,-1tr-16960875',
 'https://www.hemnet.se/bostad/lagenhet-2rum-edsberg-landsnora-sollentuna-kommun-rudbecks-vag-147-17054881',
 'https://www.hemnet.se/bostad/lagenhet-2rum-viby-sollentuna-kommun-lantgardsvagen-87-17048839',
 'https://www.hemnet.se/bostad/lagenhet-1rum-silverdal-sollentuna-kommun-margreteborgsvagen-6-17047849',
 'https://www.hemnet.se/bostad/lagenhet-2rum-centrala-sollentuna-sollentuna-kommun-turebergs-alle-41-17047447',
 'https://www.hemnet.se/bostad/lagenhet-2rum-haggvik-

In [31]:
from time import time

def decor(fn): 
    def wrapper (*args):
        t1 = time()
        func = fn(*args)
        t2 = time()
        print(f'it took {t2 - t1} seconds to run this function')
    return wrapper 

@decor
def compare_pct(*args):
    import re 
    # Remember to add new parameters here: 
    area_codes = {'järfälla_code': 17951,
                  'sollentuna_code' :18027,
                  'solna_code':18028,
                  'sundbyberg_code': 18042,
                  'stockholms_län_code': 18031,
                  'vällingby_code' : 473464, 
                  'söder_code' :898472, 
                  'bromma': 898740
                  
                 }
    
    d = dict()
    for arg in args:
        pat = re.compile(r"\b(\w*{}\w*)\b".format(arg.lower()))
        for i in area_codes.keys():
            if pat.search(i):
                d[i] = area_codes[i]
    ans = dict()
    for area,area_code in d.items():
        ans[area] = pct_change_metric(area_code = area_code, metric = 'pris_per_m2')
        
    return ans

compare_pct('solna') 

Calculating param 12m...
Calculating param 6m...
Calculating param 3m...
it took90.31149005889893 seconds to run this function


# Model deployment
1.

## 1. Flask app 

## NEXT: 

- Continue with docker https://realpython.com/python-virtual-environments-a-primer/  

- https://towardsdatascience.com/virtual-environments-104c62d48c54 
- https://www.inovex.de/blog/working-efficiently-with-jupyter-notebooks/ 




- learn more css and html
- choose the same structure and improve the design
- choose how to distribute the funtions 
- how to display tables
- Add info to the pipeline
- https://github.com/kishan0725/AJAX-Movie-Recommendation-System-with-Sentiment-Analysis
- Virtual env https://blog.testproject.io/2018/02/20/chrome-headless-selenium-python-linux-servers/

In [ ]:
######### 
#Procedure
######### 

# 1 Install miniconda https://docs.conda.io/en/latest/miniconda.html DONE!!!

# 2 Follow this guide https://www.inovex.de/blog/working-efficiently-with-jupyter-notebooks/
# 2.1 https://github.com/FlorianWilhelm/boston_housing , 
#https://towardsdatascience.com/a-guide-to-conda-environments-bc6180fc533



# 3 Test to create a project and put it on a virtual environment
#1 create a new conda env ---> install git --->  clone the repo ----> run the program using. the env
#----> install the packages ---> delete the pachages appearing in pip freeze ---> generate requirements.txt
#---> put hemnet as one folder ---> repaeat the process for data analyst 
# https://stackoverflow.com/questions/46767012/conda-list-shows-a-package-but-cannot-import-it



# 4 Uninstall anaconda https://docs.anaconda.com/anaconda/install/uninstall/

In [2]:
# import flasgger 
# from flasgger import Swagger

In [3]:
# Create a Pickle file using serialization - This step is on the notebook  

pickle_out1 = open("scraper2.pkl","wb") # this file should be on my folder 
pickle.dump(scraper2, pickle_out1) # classifier is the name of the model 

pickle_out2 = open("url_extractor.pkl","wb") 
pickle.dump(url_extractor, pickle_out2) 

pickle_out3 = open("pct_change_metric.pkl","wb") 
pickle.dump(pct_change_metric, pickle_out3) 

pickle_out1.close()
pickle_out2.close()
pickle_out3.close()

In [6]:
# 1 create the flask env
app = Flask(__name__)
#Swagger(app)

# 2 Open the file 
pickle_in1 = open("scraper2.pkl","rb")
scraper2 = pickle.load(pickle_in1)

pickle_in2 = open("url_extractor.pkl","rb")
url_extractor = pickle.load(pickle_in2)

pickle_in3 = open("pct_change_metric.pkl","rb")
pct_change_metric = pickle.load(pickle_in3)

@app.route('/')
def welcome():
    return render_template('index.html')

# @app.route('/predict',methods=["POST"])
# def predict():
    
#     """Let's predict from Hemnet 
#     This is using docstrings for specifications.
#     ---
#     parameters:  
#       - name: area
#         in: query
#         type: string
#         required: true
#       - name: keys
#         in: query
#         type: boolean
#         required: true
#       - name: min_year
#         in: query
#         type: number
#         required: true
#       - name: relevant_only
#         in: query
#         type: boolean
#         required: true
#       - name: sold_age
#         in: query
#         type: string
#         required: true
#     responses:
#         200:
#             description: The output values
        
#     """
    
#     area = request.args.get("area")
#     keys = request.args.get("keys")
#     min_year = request.args.get("min_year")
#     current_url = request.args.get("current_url")
#     relevant_only = request.args.get("relevant_only")
#     sold_age = request.args.get("sold_age")

#     current_url_new = url_extractor(area ='solletuna',keys = False, min_year = 1980 )
#     df = h_scraper(current_url = current_url_new , relevant_only = True , sold_age = '3m')
    
#     #return df.to_html(header="true", table_id="table") 
#     return render_template('index.html',prediction_text='Sales should be $ {}'.format(str(df.mean())))



@app.route('/predict',methods=["GET"])
def predict():
    
    # Remember to add new parameters here: 
    area = request.args.get("area")
    keys = request.args.get("keys")
    min_year = request.args.get("min_year")
    current_url = request.args.get("current_url")
    relevant_only = request.args.get("relevant_only")
    sold_age = request.args.get("sold_age")
    loan_limit = request.args.get("loan_limit")

    current_url_new = url_extractor(area, keys, min_year )

    df = scraper2(current_url = current_url_new ,relevant_only = relevant_only ,sold_age = sold_age,loan_limit = int(loan_limit))

    
    return render_template('view.html',tables=[df.to_html(classes ='scraper')],
    titles = ['na', 'Results'])

@app.route('/comparison',methods=["GET"])
def compare_pct():
 
    # Remember to add new parameters here: 
     
    area = request.args.get("area")
    num_pages = request.args.get("num_pages")
    metric = request.args.get("metric")

    output = pct_change_metric(area = area, num_pages = num_pages , metric = metric )

    return output

if __name__=='__main__':
    app.run()
    

# if __name__=='__main__':
#     app.run(host='0.0.0.0',port=8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2020 13:21:19] "GET / HTTP/1.1" 200 -


INITIAL EXTRACTION...
DONE WITH INITIAL EXTRACTION
STEP 1: SCRAPING...
STEP 2: GENERATING NEW FEATURES...
STEP 3: OBTAINING HISTORICAL DATA TO CALCULATE PREDICTED PRICE...


127.0.0.1 - - [07/Nov/2020 13:22:35] "GET /predict?area=solna&keys=&min_year=1990&relevant_only=yes&sold_age=3m&loan_limit=3000000 HTTP/1.1" 200 -


STEP 4: GENERATING LAST FEATURES...
DONE


## 2. Docker
https://www.youtube.com/watch?v=cDwsaQoP4Lk&list=PLZoTAELRMXVNKtpy0U_Mx9N26w8n0hIbs&index=6 

In [6]:
! pip freeze > package.txt

# GitHub pull request

In [34]:
import datetime
# 1 Get the current path 
s = ! pwd
current_path = ''
for i in s: 
    current_path+= i
# 2 Pull request
! git pull

# 3
now = datetime.datetime.now()
commit_message = "Last run on " + str(now)
! cd '/Users/Tabe/Desktop/Courses/Data-Science /Test-Projects/Python-Projects/HemnetScraper'
! echo $commit_message > commit_message.txt
! git add . 
! git commit -F commit_message.txt
! git push origin master # here I choose either master or branch 
print ('Done!')

Already up to date.
[master 3a834d1] Last run on 2020-11-01 21:49:54.742732
 Committer: Carlos De la cruz <Tabe@Carloss-MBP-2.lan>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 11 files changed, 312 insertions(+), 283 deletions(-)
 create mode 100644 HemnetScraper/pct_change_metric.pkl
Enumerating objects: 30, done.
Counting objects: 100% (29/29), done.
Delta compression using up to 4 threads
Compressing objects: 100% (15/15), done.
Writing objects: 100% (17/17), 14.57 KiB | 2.43 MiB/s, done.
Total 17 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 6 local objects.
To 